# Using Experimental Simulators with Q# and Python

The experimental simulators use the [QuTiP](https://qutip.org) library for Python to help represent noise models, so we import it here.

In [1]:
import qutip as qt

To use the experimental simulators, we start by importing Q# interoperability as normal.

In [2]:
import qsharp

Preparing Q# environment...


We can then use `qsharp.experimental.enable_noisy_simulation()` to add support for experimental simulators.

In [3]:
import qsharp.experimental
qsharp.experimental.enable_noisy_simulation()

Doing so adds the `.simulate_noise` method to Python representations of Q# callables:

In [4]:
dump_plus = qsharp.compile("""
    operation DumpPlus() : Unit {
        use q = Qubit();
        H(q);
        Microsoft.Quantum.Diagnostics.DumpMachine();
        X(q);
        Reset(q);
    }
""")

In [5]:
dump_plus.simulate_noise()

# of qubits,3
State data,$$ \left( \begin{matrix} 0.5000000000000001 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0.5000000000000001 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0.5000000000000001 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0.5000000000000001 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i\\ 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i & 0 + 0 i \end{matrix} \right) $$


()

## Configuring Open Systems Noise Models

The experimental simulators can be configured by the use of the `qsharp.config` object. For example, to change the size of the register used, we can modify the `opensim.nQubits` configuration setting:

In [6]:
qsharp.config['opensim.nQubits'] = 1

In [7]:
dump_plus.simulate_noise()

# of qubits,1
State data,$$ \left( \begin{matrix} 0.5000000000000001 + 0 i & 0.5000000000000001 + 0 i\\ 0.5000000000000001 + 0 i & 0.5000000000000001 + 0 i \end{matrix} \right) $$


()

We can modify the noise model used in simulating Q# programs by using several functions in the `qsharp.experimental` module. For instance, to initialize the noise model to an ideal model (that is, with no noise), we can use `set_noise_model_by_name`:

In [8]:
qsharp.experimental.set_noise_model_by_name('ideal')

We can then access the noise model by using `get_noise_model`:

In [9]:
noise_model = qsharp.experimental.get_noise_model()

This noise model is represented as a Python dictionary from preparations, measurements, and gates to Python objects representing the noise in each. For example, in the ideal noise model, the `Microsoft.Quantum.Intrinsic.H` operation is simulated by a unitary matrix:

In [10]:
noise_model['h']

Quantum object: dims = [[2], [2]], shape = (2, 2), type = oper, isherm = True
Qobj data =
[[ 0.70710678  0.70710678]
 [ 0.70710678 -0.70710678]]

We can modify this to add depolarizing noise using QuTiP functions to build a depolarizing noise channel:

In [11]:
I, X, Y, Z = [P.as_qobj() for P in qsharp.Pauli]

In [12]:
def depolarizing_noise(p=1.0):
    return p * qt.to_super(I) + ((1 - p) / 4) * sum(map(qt.to_super, [I, X, Y, Z]))

In [13]:
noise_model['h'] = depolarizing_noise(0.99) * qt.to_super(qt.qip.operations.hadamard_transform())

Once we have modified our noise model in this way, we can set it as the active noise model used in simulating Q# programs:

In [14]:
qsharp.experimental.set_noise_model(noise_model)

Using this model, we no longer get the exact $|+\rangle\langle+|$ state, but see that our Q# program has incurred some small error due to noise in the application of `Microsoft.Quantum.Intrinsic.H`:

In [15]:
dump_plus.simulate_noise()

# of qubits,1
State data,$$ \left( \begin{matrix} 0.5032581095356969 + 0 i & 0.4951069263733158 + 0 i\\ 0.4951069263733158 + 0 i & 0.49667422634133085 + 0 i \end{matrix} \right) $$


()

## Configuring Stabilizer Noise Models

We can also configure the experimental simulator to use stabilizer (_a.k.a._ CHP) simulation. This time, let's get a new noise model by using `get_noise_model_by_name`:

In [16]:
noise_model = qsharp.experimental.get_noise_model_by_name('ideal_stabilizer')
noise_model

{'initial_state': {'n_qubits': 1,
  'data': {'Stabilizer': array([[ True, False, False],
          [False,  True, False]])}},
 'cnot': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'Cnot': [0, 1]}]}},
 'i': {'n_qubits': 1, 'data': {'Sequence': []}},
 's': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'Phase': 0}]}},
 's_adj': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'AdjointPhase': 0}]}},
 't': {'n_qubits': 1, 'data': 'Unsupported'},
 't_adj': {'n_qubits': 1, 'data': 'Unsupported'},
 'h': {'n_qubits': 1, 'data': {'ChpDecomposition': [{'Hadamard': 0}]}},
 'x': {'n_qubits': 1,
  'data': {'ChpDecomposition': [{'Hadamard': 0},
    {'Phase': 0},
    {'Phase': 0},
    {'Hadamard': 0}]}},
 'y': {'n_qubits': 1,
  'data': {'ChpDecomposition': [{'AdjointPhase': 0},
    {'Hadamard': 0},
    {'Phase': 0},
    {'Phase': 0},
    {'Hadamard': 0},
    {'Phase': 0}]}},
 'z': {'n_qubits': 1,
  'data': {'ChpDecomposition': [{'Phase': 0}, {'Phase': 0}]}},
 'z_meas': {'pr_readout_error': 0}}

In [20]:
# TODO: not yet implemented for stabilizer models.
# qsharp.experimental.set_noise_model(noise_model)

In [21]:
qsharp.experimental.set_noise_model_by_name('ideal_stabilizer')

In [24]:
qsharp.config['opensim.representation'] = 'stabilizer'

In [25]:
dump_plus.simulate_noise()

Could not set noise model from JSON: Exception in native open systems simulator runtime: Data / schema error deserializing noise model at line 1, column 160.
JSON data:
{"initial_state":{"n_qubits":1,"data":{"Stabilizer":{"v":1,"dim":[2,3],"data":[true,false,false,false,true,false],"AsArray":[true,false,false,false,true,false]}}},"cnot":{"n_qubits":1,"data":{"ChpDecomposition":[{"Cnot":[0,1]}]}},"i":{"n_qubits":1,"data":{"Sequence":[]}},"s":{"n_qubits":1,"data":{"ChpDecomposition":[{"Phase":0}]}},"s_adj":{"n_qubits":1,"data":{"ChpDecomposition":[{"AdjointPhase":0}]}},"t":{"n_qubits":1,"data":"Unsupported"},"t_adj":{"n_qubits":1,"data":"Unsupported"},"h":{"n_qubits":1,"data":{"ChpDecomposition":[{"Hadamard":0}]}},"x":{"n_qubits":1,"data":{"ChpDecomposition":[{"Hadamard":0},{"Phase":0},{"Phase":0},{"Hadamard":0}]}},"y":{"n_qubits":1,"data":{"ChpDecomposition":[{"AdjointPhase":0},{"Hadamard":0},{"Phase":0},{"Phase":0},{"Hadamard":0},{"Phase":0}]}},"z":{"n_qubits":1,"data":{"ChpDecompositi

## Epilogue

In [ ]:
qsharp.component_versions()